## LinearSVC

In [1]:
import numpy as np
from sklearn.metrics import f1_score, recall_score, accuracy_score
from collections import defaultdict
from sklearn.svm import LinearSVC, LinearSVR
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split

### Data file path

In [66]:
data_path = "/content/drive/MyDrive/Colab Notebooks/Capstone_govt_of_canada/data/Page feedback-Vaccine pages-May17.csv"

In [67]:
df = pd.read_csv(data_path)

In [68]:
df = df[df['Tags confirmed']=='checked'][df['Lang'] == 'EN']

In [53]:
# df

In [69]:
train_df, dev_df = train_test_split(df,  test_size=0.2, random_state=11)

In [70]:
train_df
# dev_df

,Unique ID,Date,Vaccines page pairs EN/FR,Page title,URL,Comment,Tags,Refining details,Tags confirmed,Status,What's wrong,Lang
815,608c7ed08ae21611fc74e599,"30 April, 2021",Vaccine shipments EN/FR,Vaccine rollout: Vaccines and treatments for C...,https://www.canada.ca/en/public-health/service...,"Johnson and Johnson vaccine,when will it be av...",Data and tracking vaccines,Vaccine shipments and deliveries,checked,New,I can't find the information,EN
27358,6067d8328d190c273ca7e59c,"3 April, 2021",Authorized vaccines EN/FR,Authorized vaccines,https://www.canada.ca/en/health-canada/service...,Which type of vaccine used in Canada,Vaccine strategy: Authorization / Eligibility ...,Vaccine agreements / authorizations / other va...,checked,New,NaN,EN
27396,6064abf89711371dd01a1984,"31 March, 2021",Vaccines topic EN/FR,Vaccines for COVID-19,https://www.canada.ca/en/public-health/service...,Info on Johnson&Johnson vacinne,Vaccine strategy: Authorization / Eligibility ...,Vaccine agreements / authorizations / other va...,checked,New,I can't find the information,EN
1007,6089a69c8ae21611fc74d98f,"28 April, 2021",Vaccine shipments EN/FR,Vaccine rollout: Vaccines and treatments for C...,https://www.canada.ca/en/public-health/service...,update your forecasts as they are outdated,Data and tracking vaccines,Vaccine shipments and deliveries,checked,New,Other reason,EN
26527,601c1fdc6c4b8d189822fd4b,"4 February, 2021",Vaccines topic EN/FR,Vaccines for COVID-19,/content/canadasite/en/public-health/services/...,Can you drink alcohol before or after the shot,Vaccine safety (health issues / ingredients / ...,Health: pregnancy / drug interactions / who sh...,checked,New,The answer I need is missing,EN
...,...,...,...,...,...,...,...,...,...,...,...,...
4423,608c14a48ae21611fc74e2c0,"30 April, 2021",How to get vaccinated EN/FR,Vaccines for COVID-19: How to get vaccinated o...,https://www.canada.ca/en/public-health/service...,2nd dose of Pfizer vaccine. to book when. Da...,Getting vaccinated - Registration / notifications,Book / call / register for vaccines,checked,New,NaN,EN
8157,604eea2a23caed19c087fbbc,"15 March, 2021",Vaccines topic EN/FR,Vaccines for COVID-19,/content/canadasite/en/public-health/services/...,Book a vaccine,Getting vaccinated - Registration / notifications,Book / call / register for vaccines,checked,New,NaN,EN
24385,606e3c7ca2d34d23d4801e58,"7 April, 2021",Infobase: Vaccine safety EN/FR,COVID-19 vaccine safety,https://health-infobase.canada.ca/covid-19/vac...,It is unclear how to report adverse reaction t...,Vaccine safety (health issues / ingredients / ...,Side effects: possible,checked,New,The answer I need is missing,EN
4117,6092db0482338d0ecce569a5,"5 May, 2021",Vaccine shipments EN/FR,Vaccine rollout: Vaccines and treatments for C...,https://www.canada.ca/en/public-health/service...,book daughter a vacc ine,Getting vaccinated - Registration / notifications,Book / call / register for vaccines,checked,New,NaN,EN


In [71]:
train = train_df
vectorizer = CountVectorizer(ngram_range=(1,2),min_df=2)
train_texts = train.Comment.values
train_class = train.Tags.values
X_train = vectorizer.fit_transform(train_texts)
X_train

<20660x26211 sparse matrix of type '<class 'numpy.int64'>'
	with 443488 stored elements in Compressed Sparse Row format>

In [72]:
bow_df = pd.DataFrame(X_train.toarray(), columns=sorted(vectorizer.vocabulary_), index=train_texts)
bow_df

,00,00 am,00 pm,000,000 doses,000 people,000 population,01,02,03,03 2021,04,04 20,04 22,05,06,09,0f,0ver,10,10 21,10 30am,10 am,10 but,10 days,10 deaths,10 min,10 to,10 years,10 yrs,100,100 000,100 days,100 effective,1000,100th,101,102,104,104 days,...,zeneca but,zeneca can,zeneca dates,zeneca distribution,zeneca don,zeneca dose,zeneca doses,zeneca for,zeneca from,zeneca have,zeneca how,zeneca if,zeneca is,zeneca on,zeneca or,zeneca shipment,zeneca shipments,zeneca should,zeneca then,zeneca vaccine,zeneca vaccines,zeneca which,zeneca will,zeneca with,zenica,zenica doses,zenica shot,zenica vaccine,zenika,zenneca,zennica,zeno,zero,zero info,zero information,zip,zip code,zona,zone,zones
"Johnson and Johnson vaccine,when will it be available?",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Which type of vaccine used in Canada,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Info on Johnson&Johnson vacinne,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
update your forecasts as they are outdated,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Can you drink alcohol before or after the shot,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2nd dose of Pfizer vaccine. to book when. Date please.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Book a vaccine,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
It is unclear how to report adverse reaction to covid vacine.,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
book daughter a vacc ine,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [73]:
def prepare_for_classification(train,dev,max_n=2):
  '''convert lists of reviews train and dev to spare feature matrices X_train and X_test,
  and lists of polarity classifications train_class and dev_class'''
  vectorizer = CountVectorizer(ngram_range=(1,max_n),min_df=2)
  train_texts = train.Comment.values
  train_class = train.Tags.values
  dev_texts = dev.Comment.values
  dev_class = dev.Tags.values
  X_train = vectorizer.fit_transform(train_texts)
  X_dev = vectorizer.transform(dev_texts)
  return X_train,train_class, X_dev,dev_class

In [74]:
def evluate(train, test, n = 2):
    """Calculate the the kendalltau score from given train test data set, and n grams"""
    
    X_train,train_class, X_test,test_class = prepare_for_classification(train,test,max_n=n)
    clf = LinearSVC()
    clf.fit(X_train,train_class)
    
    fscore = f1_score(test_class, clf.predict(X_test), average='macro')
    acc = accuracy_score(test_class, clf.predict(X_test))
    
    print(f"{n}-gram: The shape of training set is {X_train.shape}, the fscore is {fscore}, the accuracy is {acc}")

    return fscore
    
 
print("Vaccine:")
for n in range(1, 6):
    evluate(train_df,dev_df, n)

Vaccine:


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


1-gram: The shape of training set is (20660, 4955), the fscore is 0.6899649821442608, the accuracy is 0.8060019361084221
2-gram: The shape of training set is (20660, 26211), the fscore is 0.6863941079879157, the accuracy is 0.8044530493707648
3-gram: The shape of training set is (20660, 46322), the fscore is 0.6777687271187806, the accuracy is 0.8029041626331075
4-gram: The shape of training set is (20660, 58252), the fscore is 0.6816909380930416, the accuracy is 0.8007744433688286
5-gram: The shape of training set is (20660, 64254), the fscore is 0.6811884299585025, the accuracy is 0.8
